In [1]:
import numpy as np
import pandas as pd
import re
import os

In [2]:
def readLog(reExp,logPath,logData=[]):
    '''
    빈 리스트를 인수로 받아 로그파일에서 한줄씩 읽어 추출한 데이터를 리스트에 추가하여 반환하는 함수
    
    :param reExp: 한 라인의 로그파일의 정규표현식 패턴
    :param logPath: 로그파일 경로
    :param logData: 빈 리스트
    return logData반환
    '''
    pattern = re.compile(reExp)
    with open(logPath,'r',encoding='utf8',errors='ignore') as f:
        while True:
            log = f.readline()            
            if len(log) == 0:
                break
            match = pattern.match(log)
            if match:
                logData.append([match.group(1),match.group(2),match.group(3),match.group(4),match.group(5),match.group(6)])
                
    return logData        

In [3]:
def createFileByIp(logs):
    '''
    추출한 데이타를 담은 리스트를 인수로 받아 아이피별로 파일을 생성하는 함수
    :param logs: 리스트
    :return: None
    '''
    f = None
    for log in logs:
        ip = log[2]
        if not os.path.isfile('./result/'+str(ip)+'.txt'):
            if f:
                f.close()#이전 아이피 파일객체 닫기
            f= open('./result/'+str(ip)+'.txt','w+',encoding='utf8',errors='ignore')#새로운 아이피 파일객체 생성
        
        
        f.write(f'{log[0]}\t{log[1]}\t{log[2]}\t{log[3]}\t{log[4]}\t{log[5]}\n')

In [4]:
'''
입사문제 지시 사항
생성된 파일을 읽어서 전체요청건수/메소드별 요청건수(GET/POST/PUT/DELETE)
오류율:상태가 200, 201이 아닌 나머지, 계산법 : (전체오류건수 / 전체요청건수) * 100
캐시히트율:: 캐시 결과가 TCP_HIT, TCP_IMS_HIT, TCP_REFRESH_HIT 인 경우, 계산법 : (전체캐시히트건수 / 전체요청건수) * 100 
오류율과 캐시히트율은 소수점 첫째 자리까지 반올림
summary.txt파일로 저장
정렬 하지 않고 파일로 저장
'''

def fileParsing(filePath):
    '''
    아이피.txt파일을 파싱하는 함수

    :param filePath: 아이피.txt파일의 경로
    :return: 파싱한 정보를 담은 1차원 리스트
    '''
    # 데이타 프레임으로 읽기
    df = pd.read_csv(filePath,names=['date_time','time_taken','c_ip','sc_status','s_action','cs_method'],sep='\t')
    #아이피만 문자열로 추출(filePath에서 split해도 됨)
    print('df[c_ip].unique():',df['c_ip'].unique())
    ip = df['c_ip'].unique()[0]#df['c_ip'].unique()는 ['45.108.2.47']임으로 아이피만 문자열로 추출
    txt = ip+'.txt'
    print('txt:',txt)#slashes[len(slashes)-1]와 같음
    slashes=filePath.split('/')
    print('slashes:',slashes[len(slashes)-1])
    
    total_request= len(df.index)#해당 아이피의 총 요청 건수
    get_request = len(df.query('cs_method=="GET"'))
    post_request = len(df.query('cs_method=="POST"'))
    put_request = len(df.query('cs_method=="PUT"'))
    delete_request = len(df.query('cs_method=="DELETE"'))
    
    error_count=len(df.query('not ((sc_status==200) or (sc_status==201))'))
    hit_count = len(df.query('(s_action=="TCP_HIT") or (s_action=="TCP_IMS_HIT") or (s_action=="TCP_REFRESH_HIT")'))
    
    error_rate ='%.1f' % (error_count/total_request)
    hit_rate  = '%.1f' % (hit_count/total_request)
    
    return [ip,txt,total_request,post_request,get_request,put_request,delete_request,error_rate,hit_rate]
    

In [5]:
def summarize(dirPath):
    '''
    파싱한 정보를 하나의 2차원 리스트에 저장하는 함수

    :param path: 아이피별 파일이 저장된 디렉토리 경로
    :return: 아이피별 요약정보를 담은 2차원 리스트
    '''
    files=os.listdir(dirPath)#파일 및 디렉토리명을 담은 리스트 반환
    files = [file for file in files if file.endswith('.txt')]    
    print(files)
    # 각 파일을 읽어 아이피별 요약정보를 2차원 리스트로 만들기
    summarization=[]
    for file in files:
        summarization.append(fileParsing(os.path.join(dirPath,file)))
     
    
    return summarization

In [6]:
#더미 데이타로 완료 후에는 demo_dummy.log를 demo.log
if __name__ == '__main__':
    #파일 읽기
    logData=[]
    pattern = "(\\d{4}-\\d{2}-\\d{2}\\s\\d{2}:\\d{2}:\\d{2})\\s(\\d{1,})\\s(.+)\\s(\\d{3})\\s(.+)\\s\\d{1,}\\s\\d{3,}\\s([A-Z]{3,6})\\s.+"
    logData=readLog(pattern,'demo_dummy.log',logData)
    #print(logData[0:2])
    df = pd.DataFrame(logData,columns=['date_time','time_taken','c_ip','sc_status','s_action','cs_method'])
    print(df)
    print(df.info())
    #데이타프레임을 CSV파일로 저장
    df.to_csv('demo_dummy.csv',index=False)
    #위 CSV파일로 다시 데이타 프레임 생성
    df = pd.read_csv('demo_dummy.csv')
    print(df.info())
    #아이피별로 정렬
    df.sort_values(by='c_ip',inplace=True)
    print(df)
    #cs_method별로 그룹핑
    print(df.groupby(by='cs_method').size())
    # c_ip별로 그룹핑
    print(df.groupby(by='c_ip').size())
    #총 ip는 몇개(유니크한 아이피)?
    print(len(df.groupby(by='c_ip')))
    print(len(df['c_ip'].unique()))
    #아이피별 파일 생성
    createFileByIp(df.values.tolist())#logData 와 df.values.tolist()는 같다
    print('아이피별로 파일이 생성 되었습니다')
    #파일별 즉 아이피별 요약하여 2차원 리스트로 만들기
    summarizeData = summarize('./result/')
    print('summarizeData:',summarizeData)
    #데이타 프레임으로 만든후 총 요청건수별로 정렬
    summarizeData_df = pd.DataFrame(summarizeData,columns=['ip','filename','total','post','get','put','delete','error_rate','hit_rate'])
    summarizeData_df.sort_values(by='total',inplace=True,ascending=False)
    summarizeData_df.to_csv('./result/summary.txt',header=False,index=False)
    print('요약 파일 생성 완료!!!')

             date_time time_taken          c_ip sc_status     s_action  \
0  2005-04-30 17:16:14         95  45.114.2.130       200     TCP_MISS   
1  2005-04-30 17:16:14         96  45.114.2.130       200     TCP_MISS   
2  2005-04-30 17:16:14         95  45.114.2.130       200     TCP_MISS   
3  2005-04-30 17:16:14         94  45.114.2.130       200     TCP_MISS   
4  2005-04-30 17:16:14         94  45.114.2.130       200     TCP_MISS   
5  2005-05-05 20:34:34        209   45.108.2.47       200  TCP_NC_MISS   

  cs_method  
0       GET  
1       GET  
2       GET  
3       GET  
4       GET  
5       GET  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date_time   6 non-null      object
 1   time_taken  6 non-null      object
 2   c_ip        6 non-null      object
 3   sc_status   6 non-null      object
 4   s_action    6 non-null      object
 5 